## Softmax

In [5]:
import numpy as np
import os
from gensim.models import word2vec

In [6]:
?word2vec

In [24]:
vector1 = np.array([-10, 8, -2])

In [25]:
vector2 = np.array([-1000, 1000, 1000, 1])

In [26]:
def softmax(vec):
    vec -= np.max(vec)
    exp = np.exp(vec)
    return exp / np.sum(exp)

In [27]:
softmax(vector1)

array([1.52292881e-08, 9.99954587e-01, 4.53978680e-05])

In [28]:
softmax(vector2)

array([0. , 0.5, 0.5, 0. ])

### 1.下载语料库
### 2.提取语料库
### 3.模型训练
### 4.模型使用

维基百科语料库下载：https://dumps.wikimedia.org/zhwiki/  
python WikiExtractor.py -b 200M -o zhwiki zhwiki-20190720-pages-articles-multistream.xml.bz2

In [32]:
# https://blog.csdn.net/sinat_29957455/article/details/81432846
# 合并分词后的文件
def _merge():
    output = open(r"D:\GitHub\Data\zhwiki\wiki_m", 'w', encoding='utf-8')
    input = r"D:\GitHub\Data\zhwiki\AA"
    for i in range(6):
        file_path = os.path.join(input, str("wiki_0{}".format(i)))
        file = open(file_path, 'r', encoding='utf-8')
        line = file.readline()
        while line:
            output.writelines(line)
            line = file.readline()
        file.close()
    output.close()

In [33]:
#_merge

gensim: https://radimrehurek.com/gensim/tutorial.html  
参考网站：https://www.cnblogs.com/iloveai/p/gensim_tutorial.html

解决"C extension not loaded, training will be slow."  
mingw & libpython

In [ ]:
sentences = word2vec.LineSentence(r"D:\GitHub\Data\zhwiki\wiki_m")
model = word2vec.Word2Vec(sentences, size=250)
model.save(r"D:\GitHub\Data\zhwiki\wiki_corpus.model")

## Step-05: Using visualization tools: https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne

第五步：使用Kaggle给出的T-SEN进行词向量的可视化。